In [2]:
import os
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/iamasgharrehman/DataScienceProject.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='iamasgharrehman'
os.environ['MLFLOW_TRACKING_PASSWORD']= '1b71a99195ff3f7e9c9802f1bff36ac015664d87'

In [3]:
import os
os.chdir('../')
%pwd

'd:\\Data Science\\Udemy MLOPs\\DataScienceProject'

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_path: Path
    all_params: dict
    target_column: str
    mlflow_uri: str
    

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [6]:
# Model Evaluation Config
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config= self.config.model_evaluation
        schema=self.schema.TARGET_COLUMN
        params=self.params.ElasticNet

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_path=config.metric_file_path,
            all_params=params,
            target_column=schema.name,
            mlflow_uri='https://dagshub.com/iamasgharrehman/DataScienceProject.mlflow'
        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np

In [9]:
## update Component
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self, actual, pred):
        rmse=np.sqrt(mean_squared_error(actual, pred))
        mae=mean_absolute_error(actual, pred)
        r2=r2_score(actual,pred)
        return rmse, mae, r2
    
    def log_into_mlfow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri())

        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)

            (rmse,mae,r2) = self.eval_metrics(test_y, predicted_qualities)

            # saving metrics as local
            scores = {'rmse' : rmse, 'mae': mae, 'r2' : r2}
            save_json(path=Path(self.config.metric_file_path), data=scores)


            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric('r2', r2)
            mlflow.log_metric('mae', mae)

            # Model Registry does not work with file store
            if tracking_url_type_score !='file':
                # Registor the model
                # There are other ways to use the model Registry, which depend on the use case
                # docs for more information
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")

            else:
                mlflow.sklearn.log_model(model, 'model')

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlfow()

except Exception as e:
    raise e

[2025-02-15 23:32:11,421: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-15 23:32:11,428: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-15 23:32:11,460: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-15 23:32:11,468: INFO: common: created directory at: artifacts]
[2025-02-15 23:32:11,475: INFO: common: created directory at: artifacts/model_evaluation]
[2025-02-15 23:32:12,982: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/02/15 23:32:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/02/15 23:32:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run defiant-goat-405 at: https://dagshub.com/iamasgharrehman/DataScienceProject.mlflow/#/experiments/0/runs/01c6dbe4a3cc4811a72324a5056e20d7
🧪 View experiment at: https://dagshub.com/iamasgharrehman/DataScienceProject.mlflow/#/experiments/0
